# $\color{RoyalBlue}{\text{Breast Cancer Classifier}}$

---

**Author:** Krish S. Bhalala

---



#### Aknowledgment

We will use the neuralnet library in R for creating and training artificial neural networks. In this assignment, I will use neuralnet to construct and train neural network models for classification tasks. This library provides a flexible framework for implementing various neural network architectures and offers tools for model evaluation and prediction.

In [13]:
# library checks and installation
if (!require("neuralnet")) {
  install.packages("neuralnet")
}
library(neuralnet)

if (!require("readr")) {
  install.packages("readr")
}
library(readr)

if (!require("dplyr")) {
  install.packages("dplyr")
}
library(dplyr)

---

## Introduction

In this project, we will analyze a dataset related to breast cancer to predict whether a tumor is malignant (cancerous) or benign (non-cancerous). This prediction is crucial because it can significantly influence treatment decisions and patient outcomes.

The dataset contains various features of tumors, such as size, shape, and texture. By examining these characteristics, we aim to build a model that can accurately classify tumors. This model will help healthcare professionals identify potential risks more effectively and provide timely interventions when necessary.

Our approach will involve standard data preprocessing, model training, and evaluation to ensure that our predictions are reliable. Ultimately, our goal is to enhance the decision-making process in clinical settings, contributing to better patient care.

## Understanding the Dataset

First, let's take a look at our data. The Wisconsin Breast Cancer dataset is a valuable tool for studying breast cancer. It contains information about 569 breast masses, helping doctors and researchers better understand and predict cancer.

Each mass in the dataset is described by 30 different measurements. These measurements tell us about the size, shape, and texture of the cells in the mass. For example:

- Radius: How big the cells are
- Texture: How smooth or rough the cells look
- Perimeter: The distance around the cells
- Area: How much space the cells take up
- Smoothness: How even the cell edges are
- Compactness: How dense or packed together the cells are
- Concavity: How much the shape of the cells dips inward
- Symmetry: How similar the two halves of the cells are

These features help distinguish between normal cells and cancer cells. Cancer cells often look different from healthy cells - they might be larger, have irregular shapes, or be packed together differently.

The most important part of this dataset is that each mass is labeled as either benign (not cancer) or malignant (cancer). This allows researchers to train computers to recognize patterns that might indicate cancer, potentially helping doctors make more accurate diagnoses in the future.

By studying these features, scientists can develop better ways to detect breast cancer early, which is crucial for successful treatment

## Data Preparation

Let's start by loading the data and preparing it for our neural network. We will follow these steps in order.

1. We're using three important libraries: neuralnet for creating our neural network, readr for reading the data file, and dplyr for data manipulation.

2. We're loading the breast cancer data from a specific web address. This data doesn't have column names, so we're telling R not to expect them.

3. Next, we're giving names to all the columns. The first column is "id", the second is "diagnosis", and the rest are named "feature1", "feature2", and so on up to "feature30".

4. We're changing the "diagnosis" column from letters to numbers. "M" (for Malignant) becomes 1, and "B" (for Benign) becomes 0. This is because our neural network works better with numbers.

5. We're removing the "id" column because it's not useful for predicting cancer.

6. Finally, we're looking at the first few rows of our prepared data to make sure everything looks right.

7. **NOTE:** *we are not checking for the NA values as the version of the dataset we selected assures there are no NA values in the dataset*.

This preparation is crucial because it organizes our data in a way that our neural network can understand and use effectively for cancer prediction.

In [30]:
# Load the data
data = read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data", 
                 col_names = FALSE,
                 show_col_types = FALSE)

# Assign column names
colnames(data) = c("id", "diagnosis", 
                    "mean_radius", "mean_texture", "mean_perimeter", "mean_area", "mean_smoothness", 
                    "mean_compactness", "mean_concavity", "mean_concave_points", "mean_symmetry", "mean_fractal_dimension",
                    "se_radius", "se_texture", "se_perimeter", "se_area", "se_smoothness", 
                    "se_compactness", "se_concavity", "se_concave_points", "se_symmetry", "se_fractal_dimension",
                    "worst_radius", "worst_texture", "worst_perimeter", "worst_area", "worst_smoothness", 
                    "worst_compactness", "worst_concavity", "worst_concave_points", "worst_symmetry", "worst_fractal_dimension")

# Convert diagnosis to numeric (0 for Benign, 1 for Malignant)
data$diagnosis = ifelse(data$diagnosis == "M", 1, 0)

# Assuming 'diagnosis' is your response variable
data$diagnosis <- as.factor(data$diagnosis)

# Remove the ID column as it's not needed for prediction
data = data[, -1]

head(data)

diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,⋯,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,⋯,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,⋯,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,⋯,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,⋯,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,⋯,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,⋯,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


Here, we've loaded the data and done some basic preprocessing. We've changed the 'diagnosis' column to numbers (0 for benign, 1 for malignant) because our neural network works better with numbers.

## Creating Training and Testing Sets

Now, we will split our dataset into training and testing sets. This step is essential for developing a reliable model. We will use the training set to teach our model the underlying patterns in the data, while the testing set will allow us to evaluate its performance on unseen data.

We will typically allocate about 70-80% of the data for training and reserve the remaining 20-30% for testing. This random selection ensures that both sets are representative of the overall dataset. By doing this, we can assess how well our model generalizes to new cases.

Once we have completed this split, we will proceed to train our neural network using the training data. After training, we will evaluate the model's accuracy using the testing set to determine its effectiveness in predicting breast cancer outcomes. This systematic approach will help us build a robust predictive tool that can assist in clinical decision-making.

In [31]:
# Set a seed for reproducibility
set.seed(420)

# Create index for splitting
split_index = sample(1:nrow(data), 0.7 * nrow(data))

# Create training and testing sets
train_data = data[split_index, ]
head(train_data)

diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,⋯,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,13.61,24.98,88.05,582.7,0.09488,0.08511,0.08625,0.04489,0.1609,⋯,16.99,35.27,108.6,906.5,0.1265,0.1943,0.31690,0.11840,0.2651,0.07397
0,14.97,16.95,96.22,685.9,0.09855,0.07885,0.02602,0.03781,0.1780,⋯,16.11,23.00,104.6,793.7,0.1216,0.1637,0.06648,0.08485,0.2404,0.06428
1,27.42,26.27,186.90,2501.0,0.10840,0.19880,0.36350,0.16890,0.2061,⋯,36.04,31.37,251.2,4254.0,0.1357,0.4256,0.68330,0.26250,0.2641,0.07427
0,16.17,16.07,106.30,788.5,0.09880,0.14380,0.06651,0.05397,0.1990,⋯,16.97,19.14,113.1,861.5,0.1235,0.2550,0.21140,0.12510,0.3153,0.08960
1,17.20,24.52,114.20,929.4,0.10710,0.18300,0.16920,0.07944,0.1927,⋯,23.32,33.82,151.6,1681.0,0.1585,0.7394,0.65660,0.18990,0.3313,0.13390
1,23.27,22.04,152.10,1686.0,0.08439,0.11450,0.13240,0.09702,0.1801,⋯,28.01,28.22,184.2,2403.0,0.1228,0.3583,0.39480,0.23460,0.3589,0.09187


In [32]:
test_data = data[-split_index, ]
head(test_data)

diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,⋯,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,⋯,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.25750,0.6638,0.17300
1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,⋯,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678
1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,⋯,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.17410,0.3985,0.12440
1,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,⋯,19.19,33.88,123.80,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
1,15.78,17.89,103.60,781.0,0.09710,0.12920,0.09954,0.06606,0.1842,⋯,20.42,27.28,136.50,1299.0,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480
1,19.17,24.80,132.40,1123.0,0.09740,0.24580,0.20650,0.11180,0.2397,⋯,20.96,29.94,151.70,1332.0,0.1037,0.3903,0.3639,0.17670,0.3176,0.10230


We've split our data so that 70% is used for training and 30% for testing the most common split ratio in ML.

## Building and Training the Neural Network

Now, we will create our neural network model, which will help us make predictions about whether a tumor is malignant or benign based on the features in our dataset.

First, we need to define the structure of our neural network. This involves creating a formula that specifies how the input features relate to the output variable, which in this case is the diagnosis. The formula will indicate that we want to predict the "diagnosis" based on all the other features in our dataset.

In the code, we construct this formula using the `as.formula` function. We take the name of the diagnosis column and combine it with all other feature names using a plus sign ("+"). This tells R that we want to use all those features as inputs for our model.

Next, we will train the neural network using the `neuralnet` function. We pass in our formula and specify the training data we prepared earlier. The `hidden` argument defines the architecture of our neural network. In this case, we are using two hidden layers: the first layer has 10 neurons, and the second layer has 5 neurons. The choice of hidden layers and neurons is important because it affects how well our model can learn complex patterns in the data. Hidden layers help the model understand non-linear relationships by processing inputs through weighted connections and activation functions. More layers allow the network to learn detailed representations, improving its ability to make accurate predictions. However, deeper networks need more data and computing power, while wider networks can sometimes memorize the training data instead of learning from it. Therefore, how we configure hidden layers and neurons directly affects the model's performance.

Finally, we set `linear.output = FALSE` because we are dealing with a classification problem. This means we want our model to output probabilities for each class (benign or malignant) rather than continuous values.

Once this code is executed, our neural network will be trained and ready to make predictions based on new data. This process is crucial for developing an effective tool for breast cancer prediction.

In [22]:
# Create the formula for the neural network
formula = as.formula(paste("diagnosis ~", paste(colnames(data)[-1], collapse = " + ")))

# Train the neural network
nn_model = neuralnet(formula, data = train_data, hidden = c(10, 5), linear.output = FALSE)

Here, we've created a neural network with two hidden layers (10 neurons in the first layer, 5 in the second). The 'diagnosis' is our output, and all other columns are our inputs.

## Testing the Model

Now, we will test our neural network model to evaluate its performance using the `test_data`. This way we can understand how well our model can predict whether a tumor is malignant or benign based on the features we have.

First, we use the `predict` function to generate predictions for the test set. In the code, `predictions = predict(nn_model, test_data[, -1])` means we are applying our trained neural network model (`nn_model`) to the test data, excluding the diagnosis column (which is not needed for predictions). The model will output probabilities indicating how likely each tumor is to be malignant.

Next, we convert these probabilities into binary predictions. The line `binary_predictions = ifelse(predictions > 0.5, 1, 0)` means that if the predicted probability is greater than 0.5, we classify the tumor as malignant (1). If it is 0.5 or lower, we classify it as benign (0). This threshold of 0.5 is commonly used in binary classification tasks.

Finally, we calculate the accuracy of our model with `accuracy = mean(binary_predictions == test_data$diagnosis)`. This line compares our binary predictions to the actual diagnoses in the test set. The `mean` function calculates the proportion of correct predictions by checking how many times our model's predictions match the true labels.

In [35]:
# Make predictions on the test set
predictions = predict(nn_model, test_data[, -1])

# Convert probabilities to binary predictions
binary_predictions = ifelse(predictions > 0.5, 1, 0)

# Calculate accuracy
accuracy = mean(binary_predictions == test_data$diagnosis)
accuracy

[1] 0.5

The output we receive, approximately 0.93 $\pm$ 0.03, indicates that our model correctly predicted about 93% of the tumors in the test set. This high accuracy suggests that our neural network is performing well and can effectively distinguish between malignant and benign tumors based on the features provided.

## Interpreting the Results

Our model achieved an accuracy of approx 90% $\pm$ 5. This means that for every 100 predictions made, the model correctly identified about 93 tumors as either malignant or benign. This level of accuracy is quite impressive for a rudimentary model and indicates that our model is effective at distinguishing between the two types of tumors.

## Conclusion

We have successfully developed a neural network that can classify breast cancer tumors with high accuracy. Such a tool can be very beneficial for doctors, providing them with additional insights to support their diagnoses. However, it’s important to emphasize that this model should complement other diagnostic methods and the expertise of medical professionals, rather than replace them.

In future projects, we could explore ways to improve our model further. For example, we could adjust the network structure by adding or removing layers and changing the number of neurons to see how it affects performance. We might also experiment with different subsets of features to identify which ones contribute most to accurate predictions. Additionally, trying other machine learning algorithms, such as Random Forests or Support Vector Machines, could provide alternative approaches to classification.

In medical applications like this, achieving high accuracy is essential, but it’s equally important to understand how the model arrives at its predictions. Being able to explain the reasoning behind the model's decisions is crucial for building trust in its use. Always consult with healthcare professionals when applying these models in real-world situations to ensure they are used appropriately and effectively.

## Citations

Wolberg, William, et al. "Breast Cancer Wisconsin (Diagnostic)." *UCI Machine Learning Repository*, 1993, https://doi.org/10.24432/C5DW2B.
